In [27]:
import json
import tweepy 
import pandas as pd
import requests
from bs4 import BeautifulSoup
from config import consumer_key, consumer_secret, access_token, access_token_secret
# potential library for websraping  "playwright.sync_api"
# use cases are web scraping and broswer automation, offers the
# flexibility to interact with any web content, and is not limited
# to API


auth = tweepy.OAuth1(
    consumer_key=consumer_key,
    consumer_secret=consumer_secret,
    access_token=access_token,
    access_token_secret=access_token_secret
)

# Create API object
api = tweepy.API(auth)


url = 'https://allmysportsteamssuck.com/ncaa-division-i-football-and-basketball-twitter-hashtags-and-handles/'

def scrape_team_twitter_info():
    response = requests.get(url)
    if response.status_code == 200: #okay
        soup = BeautifulSoup(response.text,"html.parser")

        # find ranks table 
        table = soup.find("table",attrs={"id":"rankingstable"})

        # parse the header
        thread = table.find("thead")

        # grab all the column names from the header
        ths = thread.find_all("th")
        col_names = [th.get_text() for th in ths]

        # parse the tbdoy, a table (2d List) of all the rows in the body
        tbody = table.find('tbody')
        trs = tbody.find_all('tr')
        rows = []
        for tr in trs:
            row = []
            tds = tr.find_all('td')
            for td in tds:
                row.append(td.get_text())
            rows.append(row)

        # make into data frame
        df = pd.DataFrame(rows,columns = col_names)
        df = df.set_index("School")
        return df
    return None # TODO: should do better error handling 


def fetch_user_account_info(client,username):
    response = client.get_user(username=username, user_fields=["created_at","description","public_metrics"])
    user = response.data
    print(user.keys())

df = scrape_team_twitter_info()
print(df.loc["Gonzaga"])

# open the file that contains the bearer token
with open("twitter_keys.json") as infile:

    # opne file and save as json obj and pull the token 
    json_obj =json.load(infile)
    token = json_obj["bearer_token"]

    # authenticating with the twitter API
    client = tweepy.Client(bearer_token=token)
zag_username = df.loc["Gonzaga","Men’s Basketball Team"][1:]
print(zag_username)
user_ser = fetch_user_account_info(client,zag_username)
print(user_ser)



AttributeError: module 'tweepy' has no attribute 'OAuth1'